In [1]:
# !pip install psycopg2

import json
from bs4 import BeautifulSoup
import re

In [2]:
import json
import pandas as pd
from bs4 import BeautifulSoup
import re

In [3]:
def clean_text(text: str) -> str:
    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text(" ", strip=True)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

processed = []

with open(r"C:/Users/DELL/OneDrive/Documents/B191333/Blackcoffer/Training_tasks/17.AI_Receptionist/AIReceptionist/blackcoffer.json", encoding="utf-8") as f:
    pages = json.load(f)

for page in pages:
    cleaned = clean_text(page["text"])
    processed.append({
        "url": page["url"],
        "title": page["title"],
        "content": cleaned
    })

In [4]:
import pandas as pd

processed_Data = pd.DataFrame(processed)
processed_Data.head(3)
processed_Data['content'][0]


'Blackcoffer provides experienced, professional statistical advice, technological soultions, consulting, information management and analysis for the enterprise and academic world. The goal of the practice is to assist people to better understand and visualize the world around them, based on an objective empirical analysis. Blackcoffer provides advanced analytic models of business processes to assist you in making optimal decisions. We truly care about our customers and our mission. Blackcoffer provide advanced analytic models of business processes to assist you in making optimal decisions. 4/2, E-Extn, Shyam Vihar Phase 1, Najafgarh, Delhi - 110043, India'

In [5]:
processed_Data.to_csv('blackcoffer_processed_data.csv', index=False)

In [6]:
processed_Data.head(3)

,url,title,content
0,https://blackcoffer.com/,"Blackcoffer - Consulting, Big Data Analytics, ...","Blackcoffer provides experienced, professional..."
1,https://blackcoffer.com/contact.html,Blackcoffer Contact us,"Blackcoffer provides experienced, professional..."
2,https://blackcoffer.com/careers.html,About Blackcoffer - Careers,"Blackcoffer provides experienced, professional..."


In [3]:
import json
import ollama
import psycopg2

conn = psycopg2.connect(
    host="db.bktcpzhlumqowbowgbme.supabase.co",
    database="postgres",
    user="postgres",
    password="jJzH34Yv3RQBGGkY",
    port=5432
)

cur = conn.cursor()
cur.execute(
    """
    create extension if not exists vector;
    
    create table if not exists documents (
      id uuid primary key default gen_random_uuid(),
      title text,
      content text,
      embedding vector(768)
    );
    """
)

conn.commit()
# for _, row in processed_Data.iterrows():
#     response = ollama.embeddings(
#         model="nomic-embed-text",
#         prompt=row["title"] + " " + row["content"]
#     )
#     embedding = response["embedding"]

#     vector_str = f"[{','.join(str(x) for x in embedding)}]"

#     cur.execute(
#         "INSERT INTO documents (title, content, embedding) VALUES (%s, %s, %s::vector);",
#         (row["title"], row["content"], vector_str)
#     )
# conn.commit()
    

In [4]:
def search_documents(query, top_k=3):
    query_emb = ollama.embeddings(
        model="nomic-embed-text",
        prompt=query
    )["embedding"]

    vector_str = f"[{','.join(str(x) for x in query_emb)}]"

    cur.execute(
        f"""
        SELECT title, content
        FROM documents
        ORDER BY embedding <-> %s::vector
        LIMIT {top_k};
        """,
        (vector_str,)
    )
    return cur.fetchall()

def generate_answer(query: str) -> str:
    docs = search_documents(query, top_k=3)

    context = "\n\n".join([f"Title: {d[0]}\n{d[1]}" for d in docs])

    prompt = f"""
You are an AI Receptionist for Blackcoffer.

Answer user queries only using the context below.
If you don't know the answer, say politely that you don’t know.

Context:
{context}

User Question: {query}
AI Receptionist Answer:
"""

    response = ollama.chat(
        model="llama3",
        messages=[{"role": "user", "content": prompt}]
    )

    return response["message"]["content"] if "message" in response else response["messages"][-1]["content"]
